In [16]:
import cv2
import numpy as np

def coarseness(image, kmax):
    image = np.array(image)
    w = image.shape[0]
    h = image.shape[1]
    kmax = kmax if (np.power(2,kmax) < w) else int(np.log(w) / np.log(2))
    kmax = kmax if (np.power(2,kmax) < h) else int(np.log(h) / np.log(2))
    average_gray = np.zeros([kmax,w,h])
    horizon = np.zeros([kmax,w,h])
    vertical = np.zeros([kmax,w,h])
    Sbest = np.zeros([w,h])

    for k in range(kmax):
        window = np.power(2,k)
        for wi in range(w)[window:(w-window)]:
            for hi in range(h)[window:(h-window)]:
                average_gray[k][wi][hi] = np.sum(image[wi-window:wi+window, hi-window:hi+window])
        for wi in range(w)[window:(w-window-1)]:
            for hi in range(h)[window:(h-window-1)]:
                horizon[k][wi][hi] = average_gray[k][wi+window][hi] - average_gray[k][wi-window][hi]
                vertical[k][wi][hi] = average_gray[k][wi][hi+window] - average_gray[k][wi][hi-window]
        horizon[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))
        vertical[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))

        
    for wi in range(w):
        for hi in range(h):
            h_max = np.max(horizon[:,wi,hi])
            h_max_index = np.argmax(horizon[:,wi,hi])
            v_max = np.max(vertical[:,wi,hi])
            v_max_index = np.argmax(vertical[:,wi,hi])
            index = h_max_index if (h_max > v_max) else v_max_index
            Sbest[wi][hi] = np.power(2,index)

    fcrs = np.mean(Sbest)
    return fcrs
def contrast(image):
    image = np.array(image)
    image = np.reshape(image, (1, image.shape[0]*image.shape[1]))
    m4 = np.mean(np.power(image - np.mean(image),4))
    v = np.var(image)
    std = np.power(v, 0.5)
    alfa4 = m4 / np.power(v,2)
    fcon = std / np.power(alfa4, 0.25)
    return fcon
def roughness(fcrs, fcon):
    return fcrs + fcon

if __name__ == '__main__':
    img = cv2.imread('dataset/OUT_2.jpg',cv2.IMREAD_GRAYSCALE)
    print(img.shape)
    fcrs = coarseness(img, 5)
    print("coarseness: %f" % fcrs);
    fcon = contrast(img)
    print("contrast: %f" % fcon)
    frgh = roughness(fcrs, fcon)
    print("roughness: %f" % frgh)

(2148, 1768)
coarseness: 9.768140
contrast: 40.879402
roughness: 50.647542
